In [3]:
import pandas as pd
import requests 
import os
from swagger_extractor import Extractor
e = Extractor("993c99861fe34c32bd0fa4b3eede2081")

In [4]:
e.qc_standard(os.getcwd())

Fetching and transforming data
Sending request for RnaSeqMetrics with data_fields: MEDIAN_CV_COVERAGE,MEDIAN_3PRIME_BIAS,MEDIAN_5PRIME_BIAS,PCT_MRNA_BASES,PCT_RIBOSOMAL_BASES,PCT_INTERGENIC_BASES,PCT_INTRONIC_BASES,PCT_CORRECT_STRAND_READS


KeyboardInterrupt: 

In [ ]:
e.out_df.index.get_level_values(1).to_list()

In [ ]:
e.out_df = e.df_list[0]

for i in range(1, len(e.df_list)):
    e.out_df = e.out_df.join(e.df_list[i], on = ["analysis_id", "sample_id"], how="inner")

In [ ]:
e.out_df

In [ ]:
sample_ids = e.final_df.index.to_list()

In [ ]:
payload = {"sample_ids":",".join(sample_ids)}

In [ ]:
r = requests.get("http://192.168.130.20:3881/sample_tracker/api/v1/sample/info", params=payload)
request_json = r.json()

In [ ]:
sample_ids = ["07df52650c634e90952b0546d86d971d",
"091387a228674865a2440b2262719ee8",
"0bf1ed33336f451a9f2129ea17500b68",
"0c5e7d205f20487f855d8ef8f39add43",
"0c9e323b3ec44af0a16001af218f0cfc",
"0d287eb3dc3348dca7b6fb10ff6395e9",
"0f023ff3259346f19f38bd11ebac3d56",
'0f9d5681deef4a09b4d2e6e766fe7c17',
"12917f77f5ea43e7ba4b1b65ae14ba5a"
]
final_json = []
request_size = 10
split_sample_ids = lambda sample_ids, request_size: [sample_ids[i:i+request_size] for i in range(0, len(sample_ids), request_size)]

for subset in split_sample_ids(sample_ids, request_size):
    payload = {"sample_ids":",".join(subset)}
    r = requests.get("http://192.168.130.20:3881/sample_tracker/api/v1/sample/info", params=payload)
    final_json.extend(r.json())

In [ ]:
final_json

In [ ]:
rdf = pd.DataFrame(final_json)
#rdf = rdf.loc[:, ["name", "uuid"]]
rdf["ann"]

In [ ]:
ye = e.final_df.join(rdf.set_index("uuid"))
ye

In [ ]:
q = pd.DataFrame(request_json)
q

In [ ]:
q = pd.json_normalize(request_json, ["data", ["items"]], ["sample_id", ["data", "row"]])

In [ ]:
q = q.loc[q["data.row"] == 1, :]

In [ ]:
q.to_excel("yeet.xlsx")

In [ ]:
pd.pivot(q.drop("data.row", axis=1), index = "sample_id", columns=["key"], values="value")

In [ ]:
pd.read_json(request_json)